In [70]:
import pandas as pd
df_label = pd.read_csv('../resource/openstack.csv', index_col=0, header=0)
df = pd.read_csv('../resource/processed_data/openstack_code_msg_2000_lines/openstack.csv', index_col=0, header=0)
df2 = pd.merge(df[['commit_id','timestamp','msg', 'codes', 'num_added', 'num_deleted', 'kinds']], df_label[['commit_id', 'buggy']])
df2.rename(columns={'buggy': 'label'}, inplace=True)
df2.to_csv('../resource/processed_data/openstack_code_msg_2000_lines/openstack_data.csv')
df2.head()

,commit_id,timestamp,msg,codes,num_added,num_deleted,kinds,label
0,00057454f6609871c2cabf874876d26a6d0d0641,1358516662,"['add lun# provider_location nexenta driver.',...",['cinder / tests / test _ nexenta . py <added_...,2,2,train,False
1,0007d255d9b20da4e5bbcdbaf5813104fbc092da,1358353428,['ensure zeros written clearing volumes note o...,['cinder / volume / drivers / lvm . py <added_...,9,6,train,False
2,005ff1100a11c83808471ece313d0f9001f4653c,1374603627,['factorize code nfs.py glusterfs.py suppresse...,['cinder / volume / drivers / glusterfs . py <...,101,165,train,False
3,006d673d24778abe6c47e3ee7e1a19e163912440,1367348740,['fix lhn driver allow backend name configurat...,['cinder / volume / drivers / san / hp _ lefth...,15,12,train,False
4,0094303e36cf1c01db539a1eea7771ebcb4d336a,1346864368,['sync improvements config file generator tool...,['tools / conf / extract _ opts . py <added_co...,143,0,train,False


In [1]:
import pandas as pd
df_label = pd.read_csv('../resource/qt.csv', index_col=0, header=0)
df = pd.read_csv('../resource/processed_data/qt_code_msg_metrics_2000_no_mail/qt.csv', index_col=0, header=0)
df2 = pd.merge(df[['commit_id','timestamp','msg', 'codes', 'num_added', 'num_deleted', 'kinds']], df_label[['commit_id', 'buggy']])
df2.rename(columns={'buggy': 'label'}, inplace=True)
df2.to_csv('../resource/processed_data/qt_code_msg_metrics_2000_no_mail/qt_data.csv')
df2.head()

,commit_id,timestamp,msg,codes,num_added,num_deleted,kinds,label
0,000025ca1ecabe363934bdc8794b01ffb926af66,1351855558,['speed tst_qlocalsocket::readbufferoverflow i...,['tests / auto / network / socket / qlocalsock...,2,0,train,False
1,0008428f9088d1d6623dec5706d49cbf65692e28,1320409350,"[""prevent timestamps widgets' touch events dis...",['src / testlib / qtesttouch . h <added_code> ...,53,39,train,False
2,0008a6f4000674020db96ff4475094d049d094f6,1381763045,"['fix compilation angle mingw broke commit .',...",['src / <num>rdparty / angle / src / libglesv<...,1,1,train,False
3,001b80305addc5e47a1c80f6629a757178d95c81,1373528887,['qdir::mkpath fail parent directory denies ac...,['src / corelib / io / qfilesystemengine _ win...,2,1,train,False
4,001bd63e813cf19d1d6abbbfeb2599e6804807d5,1311258797,['make n parameter qxxxliterals actual string ...,['src / corelib / tools / qbytearray . h <adde...,16,16,train,False


In [101]:
len(df2)

12213

In [102]:
df2[df2['kinds']== 'train'].value_counts()

commit_id                                 timestamp   input_data                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

,commit_id,timestamp,input_data,kinds,label
0,00057454f6609871c2cabf874876d26a6d0d0641,1358516662,['cinder / tests / test nexenta . py <added_co...,train,False
1,0007d255d9b20da4e5bbcdbaf5813104fbc092da,1358353428,['cinder / volume / drivers / lvm . py <added_...,train,False
2,005ff1100a11c83808471ece313d0f9001f4653c,1374603627,['cinder / volume / drivers / glusterfs . py <...,train,False
3,006d673d24778abe6c47e3ee7e1a19e163912440,1367348740,['cinder / volume / drivers / san / hp lefthan...,train,False
4,0094303e36cf1c01db539a1eea7771ebcb4d336a,1346864368,['tools / conf / extract opts . py <added_code...,train,False


In [1]:
import sys
sys.path.append('../code/')

In [54]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchtext.vocab import build_vocab_from_iterator
import numpy as np
class CNNDataset(Dataset):
    def __init__(self, data_filepath, max_lines=64, max_line_len=64, kind='train'):
        if not isinstance(data_filepath, pd.DataFrame):
            self.df = pd.read_csv(data_filepath, header=0, index_col=0)
            self.df = self.df[self.df['kinds'] == kind]
            self.df.reset_index(inplace=True, drop=True)
        else:
            self.df = data_filepath
        self.commit = self.df['commit_id']
        self.data = self.df['input_data'].apply(eval)
        self.labels = self.df['label']
        # self.code_type = ['<cpp>','<java>','<python>']
        self.max_line_len = max_line_len
        self.max_lines = max_lines
        self.corpus = self.generate_corpus()
        # self.code_type_idx = [self.corpus[code_token] for code_token in self.code_type]


    # len()を使用すると呼ばれる
    def __len__(self):
        return len(self.data)

    # 要素を参照すると呼ばれる関数    
    def __getitem__(self, idx):
        features = self.process(self.data[idx])
        features_x = torch.LongTensor(np.array(features, dtype=int))
        labels =  torch.as_tensor(int(self.labels[idx]), dtype=torch.long)
        return features_x, labels
    
    def process(self, data):
        features_list = []
        [features_list.extend(self.transform(lines)) for lines in data]
        if len(features_list) > self.max_lines:
            features_list = features_list[:self.max_lines]
        elif len(features_list) < self.max_lines:
            [features_list.extend([[self.corpus['<pad>']]*self.max_line_len]) for i in range(self.max_lines-len(features_list))]
        return features_list
    
    def transform(self, lines):
        lines2idx_list = [[self.corpus[token] for token in line.split(' ')] for line in lines]
        pad_id = self.corpus['<pad>']
        lines2idx_list2 = []
        for lines2idx in lines2idx_list:
            if lines2idx == []:
                continue
            if len(lines2idx) > self.max_line_len:
                lines2idx_list2.append(lines2idx[:self.max_line_len])
            elif len(lines2idx) < self.max_line_len:
                lines2idx += [pad_id] * (self.max_line_len-len(lines2idx))
                lines2idx_list2.append(lines2idx)
            else:
                lines2idx_list2.append(lines2idx)
        return lines2idx_list2
    
    def generate_corpus(self):
        sentences = [line.split(' ') for commit in self.data for lines in commit for line in lines if line != '']
        corpus = build_vocab_from_iterator(
            iterator=sentences,
            min_freq=1,
            specials=['<unk>', '<pad>']
        )
        corpus.set_default_index(corpus['<unk>'])
        return corpus
        

In [55]:
dataset = CNNDataset('../resource/preprocess_data/openstack_replace/openstack_data.csv')

In [56]:
dataset[0]

[[20, 14, 310, 30, 6, 9, 5, 414, 104, 5, 331, 104, 5, 143, 32, 1322, 5, 2946, 8, 58, 4, 10, 2, 225, 88, 8, 12, 7, 5, 7, 11, 2, 126, 225, 88, 5, 219, 82, 141, 8, 39, 4, 2, 225, 88, 8, 12, 11, 2, 26, 2946, 83, 42, 225, 88, 10, 2, 225, 88, 3, 210, 6, 7, 4], [20, 14, 310, 30, 6, 9, 5, 414, 104, 5, 331, 104, 5, 143, 32, 1322, 4, 10, 2, 824, 88, 8, 6, 7, 5, 7, 4, 2, 126, 824, 88, 5, 219, 82, 141, 8, 39, 4, 2, 824, 88, 8, 6, 4, 2, 126, 824, 88, 5, 219, 82, 141, 8, 39, 4, 2, 24, 9, 3, 310, 30, 6, 7, 5]]


(tensor([[ 20,  14, 310,  ...,   6,   7,   4],
         [ 20,  14, 310,  ...,   6,   7,   5],
         [  1,   1,   1,  ...,   1,   1,   1],
         ...,
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1],
         [  1,   1,   1,  ...,   1,   1,   1]]),
 tensor(0))

In [57]:
len(dataset.corpus)

6378

In [89]:
class CNNDataset(Dataset):
    def __init__(self, data_filepath, spm_modelpath=None, max_lines=64, max_line_len=32, kind='train'):
        self.df = pd.read_csv(data_filepath, header=0, index_col=0)
        self.corpus = self.generate_corpus(self.df[self.df['kinds'] == 'train']['input_data'].apply(eval))
        self.df = self.df[self.df['kinds'] == kind]
        self.df.reset_index(inplace=True, drop=True)
        self.commit = self.df['commit_id']
        self.data = self.df['input_data'].apply(eval)
        self.labels = self.df['label']
        # self.code_type = ['<cpp>','<java>','<python>']
        self.max_line_len = max_line_len
        self.max_lines = max_lines
        
        # self.code_type_idx = [self.corpus[code_token] for code_token in self.code_type]


    # len()を使用すると呼ばれる
    def __len__(self):
        return len(self.data)

    # 要素を参照すると呼ばれる関数    
    def __getitem__(self, idx):
        features = self.process(self.data[idx])
        features_x = torch.LongTensor(np.array(features, dtype=int))
        labels =  torch.as_tensor(int(self.labels[idx]), dtype=torch.long)
        return features_x, labels
    
    def process(self, data):
        features_list = [self.transform(line) for line in data]
        if len(features_list) > self.max_lines:
            features_list = features_list[:self.max_lines]
        elif len(features_list) < self.max_lines:
            [features_list.extend([[self.corpus['<pad>']]*self.max_line_len]) for i in range(self.max_lines-len(features_list))]
        return features_list
    
    def transform(self, line):
        lines2idx = [self.corpus[token] for token in line.split(' ')]
        pad_id = self.corpus['<pad>']
        if len(lines2idx) > self.max_line_len:
            lines2idx = lines2idx[:self.max_line_len]
        elif len(lines2idx) < self.max_line_len:
            lines2idx += [pad_id] * (self.max_line_len-len(lines2idx))
        return lines2idx
    
    def generate_corpus(self, data):
        # print(data)
        # for lines in data:
        #     print(lines)
        #     for line in lines:
        #         print(line)
        sentences = [line.split(' ') for lines in data for line in lines if line != '']
        corpus = build_vocab_from_iterator(
            iterator=sentences,
            min_freq=1,
            specials=['<unk>', '<pad>']
        )
        corpus.set_default_index(corpus['<unk>'])
        return corpus

In [98]:
dataset = CNNDataset('../resource/preprocess_data/openstack_after_replace_no_space/openstack_data.csv')

In [93]:
for i in range(len(dataset)):
    dataset[i]

In [64]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchtext.vocab import build_vocab_from_iterator
import numpy as np
from torch.utils.data import Dataset
class CNNDataset(Dataset):
    def __init__(self, data_filepath, spm_modelpath=None, max_lines=64, max_line_len=32, kind='train'):
        self.df = pd.read_csv(data_filepath, header=0, index_col=0)
        self.corpus = self.generate_corpus(self.df[self.df['kinds'] == 'train']['input_data'].apply(eval))
        self.df = self.df[self.df['kinds'] == kind]
        self.df.reset_index(inplace=True, drop=True)
        self.commit = self.df['commit_id']
        self.data = self.df['input_data'].apply(eval)
        self.labels = self.df['label']
        # self.code_type = ['<cpp>','<java>','<python>']
        self.max_line_len = max_line_len
        self.max_lines = max_lines
        
        # self.code_type_idx = [self.corpus[code_token] for code_token in self.code_type]


    # len()を使用すると呼ばれる
    def __len__(self):
        return len(self.data)

    # 要素を参照すると呼ばれる関数    
    def __getitem__(self, idx):
        features, segment = self.process(self.data[idx])
        features_x = torch.LongTensor(np.array(features, dtype=int))
        segment = torch.LongTensor(np.array(segment, dtype=int))
        labels =  torch.as_tensor(int(self.labels[idx]), dtype=torch.long)
        return features_x, segment, labels
    
    def process(self, data):
        features_list = []
        segments_list = []
        data_list = [features_list, segments_list]
        [data_list[i].append(result) for line in data for i, result in enumerate(self.transform(line))]
        if len(features_list) > self.max_lines:
            features_list = features_list[:self.max_lines]
            segments_list = segments_list[:self.max_lines]
        elif len(features_list) < self.max_lines:
            features_list.extend([[self.corpus['<pad>']]*self.max_line_len for i in range(self.max_lines-len(features_list))])
            segments_list.extend([[0]*self.max_line_len for i in range(self.max_lines-len(segments_list))])
        return features_list, segments_list
    
    def transform(self, line):
        lines2idx = [self.corpus[token] for token in line.split(' ')]
        pad_id = self.corpus['<pad>']
        added_code_id = self.corpus['<added_code>']
        deleted_code_id = self.corpus['<deleted_code>']
        pos_id = np.where((np.array(lines2idx) == added_code_id) | (np.array(lines2idx) == deleted_code_id))[0][0]
        segment_label = np.ones(len(lines2idx))
        segment_label[pos_id:] = 2
        
        if len(lines2idx) > self.max_line_len:
            lines2idx = lines2idx[:self.max_line_len]
            segment_label = segment_label[:self.max_line_len]
        elif len(lines2idx) < self.max_line_len:
            lines2idx += [pad_id] * (self.max_line_len-len(lines2idx))
            segment_label = np.concatenate([segment_label, np.zeros(self.max_line_len-len(segment_label))])
        return lines2idx, segment_label
    
    def generate_corpus(self, data):
        sentences = [line.split(' ') for lines in data for line in lines if line != '']
        corpus = build_vocab_from_iterator(
            iterator=sentences,
            min_freq=3,
            specials=['<unk>', '<pad>']
        )
        corpus.set_default_index(corpus['<unk>'])
        return corpus

In [65]:
dataset = CNNDataset('../resource/preprocess_data/openstack_no_tokenize/openstack_data.csv')

In [68]:
len(dataset[2][1])

2


266